In [1]:
import pandas as pd 

In [2]:
year = '2016'

In [3]:
df = pd.read_csv('data/points_against_'+year+'.csv')

In [4]:
# Calculate Team Against Ave/Std
for pos in df.Position.unique():
    for team in df.Team_Against.unique():
        num_weeks = df[(df.Team_Against == team) & (df.Position == pos)].Game_Count.sum()
        if num_weeks != 0:
            ave = df[(df.Team_Against == team) & (df.Position == pos)].Points_Against.astype(float).sum() / num_weeks
        else:
            print(df[(df.Team_Against == team) & (df.Position == pos)])
        std = df[(df.Team_Against == team) & (df.Position == pos)].Points_Against.astype(float).std()
        df.loc[(df.Team_Against == team) & (df.Position == pos), 'Against_Ave'] = ave
        df.loc[(df.Team_Against == team) & (df.Position == pos), 'Against_Std'] = std
        df['Against_16th'] = df['Against_Ave'] - df['Against_Std']
        df['Against_84th'] = df['Against_Ave'] + df['Against_Std']

In [5]:
# Calculate Deviation from average
df['Game_Dev'] = (df.Points_Against - df.Against_Ave)/(df.Against_Std)

In [6]:
# Construct Summary DF from the 'VS' team perspective
df_vs = df[['VS','Position','Year']]
df_vs = df_vs.drop_duplicates()
df_vs = df_vs[df_vs.VS != 'Bye']

In [7]:
# Calculate Ave/Std (of deviations) for the VS teams
df_vs['VS_Ave'] = 0
df_vs['VS_Std'] = 0
for index, row in df_vs.iterrows():
    df_vs.loc[(df_vs['VS'] == row.VS) & (df_vs.Position == row.Position) & (df_vs.Year == row.Year),'VS_Ave'] = df[(df.VS == row.VS) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.mean()
    df_vs.loc[(df_vs['VS'] == row.VS) & (df_vs.Position == row.Position) & (df_vs.Year == row.Year),'VS_Std'] = df[(df.VS == row.VS) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.std()

In [8]:
# Predict scores in a matchup
def matchup(team_against, vs):
    df_against = df[(df['Team_Against'] == team_against)][['Team_Against','Position','Against_Ave','Against_Std','Against_16th','Against_84th']]
    df_against = df_against.drop_duplicates()
    
    df_matchup = pd.merge(left=df_against, right=df_vs[df_vs['VS']==vs], left_on='Position', right_on='Position')
    
    df_matchup['Weighted_16th'] = (df_matchup['VS_Ave']-df_matchup['VS_Std'])*df_matchup['Against_Std']+df_matchup['Against_Ave']
    df_matchup['Weighted_Mean'] = (df_matchup['VS_Ave'])*df_matchup['Against_Std']+df_matchup['Against_Ave']
    df_matchup['Weighted_84th'] = (df_matchup['VS_Ave']+df_matchup['VS_Std'])*df_matchup['Against_Std']+df_matchup['Against_Ave']
    
    return df_matchup
    

In [9]:
matchup('PIT', 'CHI')

,Team_Against,Position,Against_Ave,Against_Std,Against_16th,Against_84th,VS,Year,VS_Ave,VS_Std,Weighted_16th,Weighted_Mean,Weighted_84th
0,PIT,QB,14.62125,5.127092,9.494158,19.748342,CHI,2016,-0.488994,0.693182,8.560127,12.114134,15.668142
1,PIT,RB,20.73750,13.329677,7.407823,34.067177,CHI,2016,-0.097576,0.595532,11.498594,19.436846,27.375098
2,PIT,WR,19.00000,7.785342,11.214658,26.785342,CHI,2016,0.109760,0.925662,12.647919,19.854517,27.061116
3,PIT,TE,7.61250,4.287619,3.324881,11.900119,CHI,2016,-0.480099,0.758528,2.301740,5.554019,8.806297
4,PIT,K,6.81250,4.416713,2.395787,11.229213,CHI,2016,-0.597555,0.574291,1.636793,4.173272,6.709752
5,PIT,DEF,3.87500,4.554410,-0.679410,8.429410,CHI,2016,-0.130321,1.065035,-1.569139,3.281464,8.132068


In [10]:
schedule_year = 2017
df_schedule = pd.read_csv('data/schedule_'+str(schedule_year)+'.csv')
def get_week(wk):
    df_tmp = df_schedule[df_schedule['Week'] == wk]
    
    df_tmp['Team1'] = df_tmp['Away']
    df_tmp['Team2'] = df_tmp['Home']
    df_week = pd.DataFrame()
    df_week['Team1'] = df_tmp['Home']
    df_week['Team2'] = df_tmp['Away']
    df_week = pd.concat([df_tmp,df_week])[['Team1','Team2']]
    return df_week

In [11]:
wk = 3

In [12]:
df_week = get_week(3)
df_wk_matchups = pd.DataFrame()
for index,row in df_week.iterrows():
    df_tmp = matchup(row.Team1, row.Team2)[['VS','Team_Against','Position','Weighted_16th','Weighted_Mean','Weighted_84th']]
    df_wk_matchups = pd.concat([df_wk_matchups,df_tmp])

/Users/justin/anaconda3/envs/p3-ml/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/justin/anaconda3/envs/p3-ml/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [13]:
df_wk_matchups[df_wk_matchups.Position=='QB']

,VS,Team_Against,Position,Weighted_16th,Weighted_Mean,Weighted_84th
0,BAL,JAX,QB,10.300286,14.080472,17.860658
0,CLE,IND,QB,10.301741,15.372449,20.443157
0,NYG,PHI,QB,6.620329,12.445311,18.270292
0,MIA,NYJ,QB,8.585549,15.688032,22.790514
0,DEN,BUF,QB,5.506868,12.622193,19.737518
0,NO,CAR,QB,12.633525,21.636518,30.639511
0,PIT,CHI,QB,10.354803,18.513489,26.672175
0,ATL,DET,QB,18.744663,25.321142,31.897621
0,TB,MIN,QB,8.841875,14.463328,20.084781
0,HOU,NE,QB,6.872492,10.786869,14.701246
